In [ ]:
!pip install opensea
import requests
import pandas as pd
import json
import random
import datetime
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Get events that happened in 2022

In [ ]:
asset_events = [] #creating a list? 
url = "https://api.opensea.io/api/v1/events"
headers = {"Accept": "application/json",
"X-API-KEY": "78d7597e1ad24043937e877bd04e1e66"}

begindate = datetime.datetime.strptime('2022-01-01','%Y-%m-%d')
enddate = datetime.datetime.strptime('2022-04-30','%Y-%m-%d')
#it collects 50 assets from everyday between begindate and enddate
while begindate<enddate:
    print(begindate)
    #rint = random.randint(1,364)
    date1 = begindate
    date2 = begindate + datetime.timedelta(days=1)
    date1 = datetime.datetime.strftime(date1,'%Y-%m-%d')
    date2 = datetime.datetime.strftime(date2,'%Y-%m-%d')
    url = 'https://api.opensea.io/api/v1/events?occurred_before=' + date2 + 'T00%3A00%3A00&occurred_after=' + date1 + 'T00%3A00%3A00'
    params = {'limit':'50','event_type':'successful'}

    response = requests.get(url, headers=headers,params=params)
    print(response)
    x = response.json()
    asset_events = asset_events + x['asset_events']
    begindate = begindate + datetime.timedelta(days=1)

In [ ]:
asset_events_df = pd.json_normalize(asset_events)

In [ ]:
asset_events_df

In [ ]:
asset_events_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/events_keys.csv',index=False)

In [ ]:
#cleaning name column fields to erase punctuation and digits so it works better with Twitter's snscrape 
asset_events_df['asset.name'] = asset_events_df['asset.name'].str.replace(r'[^\w\s]+', '')
asset_events_df['asset.name'] = asset_events_df['asset.name'].str.replace('\d+', '')
asset_events_df['asset.name'] = asset_events_df['asset.name'].str.replace(' ', '')

In [ ]:
asset_events_df.info(verbose = True)

In [ ]:
#cleaning total price, transform from wei to eth and turn total price to float 
asset_events_df['total_price'] = asset_events_df['total_price'].astype(float, errors = 'raise')
asset_events_df['total_price'] = asset_events_df['total_price']/10.**18

In [ ]:
asset_events_df['quantity'] = pd.to_numeric(asset_events_df['quantity'],errors = 'coerce')

In [ ]:
asset_events_df['payment_token.usd_price'] = pd.to_numeric(asset_events_df['payment_token.usd_price'],errors = 'coerce')

In [ ]:
#Calculating sales price in USD. For the ones in NaN can I replace by the same value the rest of them have? or an average?  
asset_events_df['total_price_usd'] = asset_events_df['total_price'] * asset_events_df['payment_token.usd_price']

In [ ]:
asset_events_df['total_price_usd']

In [ ]:
asset_events_df = asset_events_df[['total_price','quantity','collection_slug','asset.num_sales','asset.name','payment_token.symbol','payment_token.usd_price','asset.asset_contract.address', 'asset.token_id', 'dev_seller_fee_basis_points']]

In [ ]:
asset_events_df

In [ ]:
asset_events_df[asset_events_df['payment_token.symbol'].isna()]#1957 rows 

In [ ]:
asset_events_df['payment_token.symbol'].isna().sum()

1970

In [ ]:
asset_events_df['payment_token.usd_price'].mean()#maybe I could use this for missing values? 

1750.7427269932894

In [ ]:
asset_events_df['payment_token.symbol'].unique()

array([nan, 'ETH', 'WETH', 'ASH', 'GALA', 'CUBE'], dtype=object)

In [ ]:
asset_events_eth_df = asset_events_df.loc[(asset_events_df['payment_token.symbol'] == 'ETH') | (asset_events_df['payment_token.symbol'] =='WETH')]

In [ ]:
#Calculating sales price in USD. For the ones in NaN can I replace by the same value the rest of them have? or an average?  
asset_events_eth_df['total_price_usd'] = asset_events_eth_df['total_price'] * asset_events_eth_df['payment_token.usd_price']

In [ ]:
asset_events_eth_df

,total_price,quantity,collection_slug,asset.num_sales,asset.name,payment_token.symbol,payment_token.usd_price,asset.asset_contract.address,asset.token_id,dev_seller_fee_basis_points,total_price_usd
2,0.7500,1,alienfrensnft,5.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,6852,700.0,893.715000
3,0.7500,1,alienfrensnft,5.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,6374,700.0,893.715000
4,0.7490,1,alienfrensnft,4.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,419,700.0,892.523380
5,0.7440,1,alienfrensnft,7.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,8812,700.0,886.565280
6,0.7400,1,alienfrensnft,3.0,alienfren,ETH,1191.62,0x123b30e25973fecd8354dd5f41cc45a3065ef88c,7210,700.0,881.798800
...,...,...,...,...,...,...,...,...,...,...,...
5939,0.0800,1,mutantshibaclub,1.0,MutantShiba,ETH,1191.62,0xca95777971c7c438eecbd8bfc23249fbee99e716,6906,800.0,95.329600
5940,0.0040,1,decentrabis,1.0,DecentrabisNuggy,ETH,1191.62,0x6b7b0bba4b8c26680f267f5621025f146a4cbddf,2447,500.0,4.766480
5941,0.0532,1,regulars,5.0,Regular,ETH,1191.62,0x6d0de90cdc47047982238fcf69944555d27ecb25,2209,500.0,63.394184
5942,0.0900,1,e-shell,1.0,EShell,ETH,1191.62,0xe41bea6888f771a0c16d7188284522b76c135252,2061,750.0,107.245800


In [ ]:
asset_events_eth_df = asset_events_eth_df.groupby(['asset.name','collection_slug'], as_index=False).agg({'total_price': 'mean', 'quantity': 'first','asset.num_sales':'mean', 'total_price_usd':'first', 'payment_token.usd_price':'mean', 'total_price_usd':'mean' })

In [ ]:
#Agreggate by asset name and collection slug 
asset_events_eth_df = asset_events_eth_df.groupby(['asset.name','collection_slug'], as_index = False).sum()

In [ ]:
asset_events_eth_df.keys()

Index(['total_price', 'quantity', 'collection_slug', 'asset.num_sales',
       'asset.name', 'payment_token.symbol', 'payment_token.usd_price',
       'asset.asset_contract.address', 'asset.token_id',
       'dev_seller_fee_basis_points', 'total_price_usd'],
      dtype='object')

In [ ]:
len(asset_events_eth_df)#we have 5950 events

1530

In [ ]:
len(asset_events_eth_df['asset.name'].unique())# I am doing this to check that we don't have a lot of repeated ones 
#we have 2341 unique asset events 

1417

In [ ]:
asset_events_eth_df

,asset.name,collection_slug,total_price,quantity,asset.num_sales,total_price_usd,payment_token.usd_price
0,,animo-official,0.159450,1,4.000000,190.003809,1191.620
1,,avius-animae,0.042000,1,1.000000,50.048040,1191.620
2,,baby-cub-tigers,0.060000,1,2.000000,71.283000,1188.050
3,,bakedape,0.250000,1,1.000000,297.905000,1191.620
4,,barn-owlz,0.060000,1,3.000000,71.497200,1191.620
...,...,...,...,...,...,...,...
1523,xmfer,xmfers,0.186610,1,2.200000,222.261108,1191.263
1524,xmfers,0xmfers,0.003267,1,1.333333,3.892625,1191.620
1525,Ïmumonen,lootrealms,1.750000,1,3.000000,2085.335000,1191.620
1526,ἀκτῑς,futuristek-hieroglyphx,0.077700,1,21.000000,92.588874,1191.620


In [ ]:
asset_events_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/asset_events.csv',index=False)

In [ ]:
asset_events_eth_df.to_csv('/content/drive/MyDrive/MADSThesis_Alejandro/eth_events_2.csv',index=False)

In [ ]:
asset_events_df = pd.read_csv('/content/drive/MyDrive/MADSThesis_Alejandro/asset_events.csv')